# UPGMA Algorithm

1. Assign each taxon to its own cluster

2. Define one leaf for each taxon; place it at height 0

3. While more than two clusters:

    - Determine two clusters $i$ and $j$ with smallest $d_{ij}$
    
    - Define a new cluster $C_k = C_i ∪ C_j$

    - Define a node $k$ with children $i$ and $j$, place it at height $d_{ij}/2$
    
    - Replace clusters $i$ and $j$ with $k$
    
    - Compute distance between $k$ and other clusters
    
4. Join last two clusters $i$ and $j$ by root at height $d_{ij}/2$

In [1]:
import numpy as np

In [2]:
# Input matrices must be square and symmetric with -1s on the diagonal

example = [
    [-1, 19, 27, 8, 33, 18, 13],
    [19, -1, 31, 18, 36, 1, 13],
    [27, 31, -1, 26, 41, 32, 29],
    [8, 18, 26, -1, 31, 17, 14],
    [33, 36, 41, 31, -1, 35, 28],
    [18, 1, 32, 17, 35, -1, 12],
    [13, 13, 29, 14, 28, 12, -1]
]

In [ ]:
# example = {
#     0: [],
#     1: [19],
#     2: [27, 31],
#     3: [8, 18, 26],
#     4: [33, 36, 41, 31],
#     5: [18, 1, 32, 17, 35],
#     6: [13, 13, 29, 14, 28, 12]
# }

In [18]:
def upgma_iteration(matrix, original, pair_list):
    """
    matrix = square matrix input
    original = the first matrix input
    """
    original = np.array(matrix, dtype=float)

    # Convert matrix input to numpy array
    matrix = np.array(matrix, dtype=float)
    dim = matrix.shape

    # Check to make sure matrix size is valid for the algorithm
    assert(dim[0] == dim[1]), "Input is not a square matrix."
    assert(dim[0 != 1]), "Everything is grouped."

    # Find the next shortest pairwise distance and the corresponding indices (groups) in the CURRENT matrix
    current_min = np.max(matrix)
    pair = (-1,-1)
    for r in range(dim[0]):
        for c in range(dim[1]):
            if r <= c: #stay out of upper traingular matix
                continue
            elif current_min > matrix[r,c]:
                current_min = matrix[r,c]
                pair = (r, c)

    
    pair_current = pair # in context of current matrix

    # Reconstruct the original values of the indices of pair so that they fit into the original matrix (we will need to use the pair list)
    pair_original = pair # TODO: in context of original matrix (this is what we need to reconstruct)
    print(pair_original)
    
    # Add to pair list
    pair_list.append(pair_original)

    # Calculate the depth of the new branch
    depth = current_min / 2

    slice1 = matrix[:, pair[0]]
    slice1[pair[1]] = -1
    slice2 = matrix[:, pair[1]]
    slice2[pair[0]] = -1

    # Calculate the mean pairwise distances with other sequences in new matrix.
    pairwise_mean = sum([slice1, slice2]) / 2 #(len(pair_list)*2)

    print(pairwise_mean)
    # Remove the second occurence of -1
    second_negative_index = np.where(pairwise_mean==-1)[0][1]
    pairwise_mean = np.delete(pairwise_mean, second_negative_index)

    # Construct the new matrix
    index_del = max(pair_current)
    index_replace = min(pair_current)
    new_matrix = np.delete(matrix, index_del, axis=0)
    new_matrix = np.delete(new_matrix, index_del, axis=1)

    for i in range(len(pairwise_mean)):
        new_matrix[index_replace,i] = pairwise_mean[i]
        new_matrix[i,index_replace] = pairwise_mean[i]

    new_matrix[new_matrix == 0] = -1
    
    return {"new_matrix":new_matrix, "pair":pair_original, "depth":depth, "original_matrix": original}

In [19]:
def upgma_full(matrix):

    original = np.array(matrix, dtype=float)
    current_matrix = np.array(matrix, dtype=float)
    dim = current_matrix.shape[0]
    pairing_stack = [] # keep track of prior pairs so that we can reconstruct the matrix later
    pair_list = [] # keep track of indices of the prior minimum in the original matrix
    
    while dim > 1:
        iterresults = upgma_iteration(current_matrix, original, pair_list)
        pairing_stack.append(iterresults)
        current_matrix = iterresults["new_matrix"]
        dim = current_matrix.shape[0]


In [20]:
stack = upgma_full(example)

(5, 1)
[array([18., -1., 32., 17., 35., -1., 12.]), array([19., -1., 31., 18., 36., -1., 13.])]
[18.5 -1.  31.5 17.5 35.5 -1.  12.5]
(3, 0)
[array([13., -1., 29., 14., 28., -1.]), array([18.5, -1. , 31.5, 17.5, 35.5, -1. ]), array([-1. , 17.5, 26. , -1. , 31. , 14. ]), array([-1. , 18.5, 27. , -1. , 33. , 13. ])]
[ 7.375  8.5   28.375  7.375 31.875  6.25 ]


IndexError: index 1 is out of bounds for axis 0 with size 0